This notebook uses an SGCN module in the bispy package to search the National List SGCN API. It returns and caches the summarized National List records, which include the list of states that have the species in their conservation planning process.

In [5]:
#Import needed packages
import bispy
import json
from IPython.display import display
from joblib import Parallel, delayed

sgcn = bispy.sgcn.Search()
bis_utils = bispy.bis.Utils()

In [6]:
# Open up cache updated ITIS list with species names from ITIS Exploration.ipynb
with open("../cache/itis_explore.json", "r") as f:
    itis_explore = json.loads(f.read())

In [7]:
# Use joblib to run multiple requests for SGCN species records in parallel via ITIS scientific names
sgcn_result = Parallel(n_jobs=8)(delayed(sgcn.search)(name) for name in [r["scientific_name"] for r in itis_explore]) 

In [8]:
#Display result
sgcn_result

[{'processing_metadata': {'status': 'failure',
   'date_processed': '2019-09-27T20:18:56.433306',
   'status_message': 'Not Matched',
   'api': 'https://api.sciencebase.gov/bis-api/api/v1/swap/nationallist?scientificname=Otis tarda'},
  'parameters': {'Scientific Name': 'Otis tarda', 'Name Source': None}},
 {'processing_metadata': {'status': 'success',
   'date_processed': '2019-09-27T20:18:56.426361',
   'status_message': 'Name Match',
   'api': 'https://api.sciencebase.gov/bis-api/api/v1/swap/nationallist?scientificname=Clangula hyemalis'},
  'parameters': {'Scientific Name': 'Clangula hyemalis', 'Name Source': None},
  'data': {'statelist_2005': 'Alaska,Connecticut,Delaware,Massachusetts,New Jersey,New York,Washington',
   'statelist_2015': 'Alaska,Delaware,Maine,Maryland,Massachusetts,New Jersey,New York,North Carolina,Pennsylvania,Washington',
   'scientificname': 'Clangula hyemalis',
   'commonname': 'Long-tailed Duck',
   'taxonomicgroup': 'Birds',
   'taxonomicrank': 'Species',

In [9]:
# Filter to give just cases where SGCN species names matched with ITIS species names
success_sgcn=[i for i in sgcn_result if i['processing_metadata']['status'] == 'success']
success_sgcn

[{'processing_metadata': {'status': 'success',
   'date_processed': '2019-09-27T20:18:56.426361',
   'status_message': 'Name Match',
   'api': 'https://api.sciencebase.gov/bis-api/api/v1/swap/nationallist?scientificname=Clangula hyemalis'},
  'parameters': {'Scientific Name': 'Clangula hyemalis', 'Name Source': None},
  'data': {'statelist_2005': 'Alaska,Connecticut,Delaware,Massachusetts,New Jersey,New York,Washington',
   'statelist_2015': 'Alaska,Delaware,Maine,Maryland,Massachusetts,New Jersey,New York,North Carolina,Pennsylvania,Washington',
   'scientificname': 'Clangula hyemalis',
   'commonname': 'Long-tailed Duck',
   'taxonomicgroup': 'Birds',
   'taxonomicrank': 'Species',
   'matchmethod': 'Exact Match',
   'acceptedauthorityurl': 'https://www.itis.gov/servlet/SingleRpt/SingleRpt?search_topic=TSN&search_value=175147'}},
 {'processing_metadata': {'status': 'success',
   'date_processed': '2019-09-27T20:18:56.027081',
   'status_message': 'Name Match',
   'api': 'https://api.

In [10]:
# Cache the array of retrieved documents and return/display a random sample for verification
display(bis_utils.doc_cache("../cache/sgcn.json", success_sgcn))

{'Doc Cache File': '../cache/sgcn.json',
 'Number of Documents in Cache': 98,
 'Document Number 38': {'processing_metadata': {'status': 'success',
   'date_processed': '2019-09-27T20:18:59.379365',
   'status_message': 'Name Match',
   'api': 'https://api.sciencebase.gov/bis-api/api/v1/swap/nationallist?scientificname=Ictidomys tridecemlineatus'},
  'parameters': {'Scientific Name': 'Ictidomys tridecemlineatus',
   'Name Source': None},
  'data': {'statelist_2005': 'Arizona,Missouri,Ohio,Utah',
   'statelist_2015': 'Arizona,Missouri,Ohio',
   'scientificname': 'Ictidomys tridecemlineatus',
   'commonname': 'Thirteen-lined Ground Squirrel',
   'taxonomicgroup': 'Mammals',
   'taxonomicrank': 'Species',
   'matchmethod': 'Followed Accepted TSN',
   'acceptedauthorityurl': 'https://www.itis.gov/servlet/SingleRpt/SingleRpt?search_topic=TSN&search_value=930308'}}}

Here we look to see which WLCI species names show up on the Wyoming SGCN list

In [13]:
#Create list of species names that appear on the Wyoming 2005 and/or 2015 SGCN state list along with additional species information
wyoming_sgcn=[]
for rec in [r["data"] for r in sgcn_result if r["processing_metadata"]["status"] == "success"]:
    scietific_name=rec['scientificname']
    common_name=rec['commonname']
    list_2005=rec['statelist_2005']
    list_2015=rec['statelist_2005']
    match_method=rec['matchmethod']
    taxonomic_group=rec['taxonomicgroup']
    if "Wyoming" in rec["statelist_2005"] or "Wyoming" in rec["statelist_2015"]:
        wyoming_sgcn.append({'scientificname': scietific_name,'commonname':common_name,'statelist_2005':list_2005, 'statelist_2015':list_2015, 'matchmethod':match_method, 'taxonomicgroup':taxonomic_group })

In [14]:
#Display list
wyoming_sgcn

[{'scientificname': 'Artemisiospiza belli',
  'commonname': "Bell's Sparrow",
  'statelist_2005': 'Colorado,Montana,Nevada,New Mexico,Oregon,Utah,Washington,Wyoming',
  'statelist_2015': 'Colorado,Montana,Nevada,New Mexico,Oregon,Utah,Washington,Wyoming',
  'matchmethod': 'Followed Accepted TSN',
  'taxonomicgroup': 'Birds'},
 {'scientificname': 'Buteo regalis',
  'commonname': 'Ferruginous Hawk',
  'statelist_2005': 'Arizona,California,Colorado,Idaho,Kansas,Montana,Nebraska,Nevada,New Mexico,North Dakota,Oklahoma,Oregon,South Dakota,Texas,Utah,Washington,Wyoming',
  'statelist_2015': 'Arizona,California,Colorado,Idaho,Kansas,Montana,Nebraska,Nevada,New Mexico,North Dakota,Oklahoma,Oregon,South Dakota,Texas,Utah,Washington,Wyoming',
  'matchmethod': 'Exact Match',
  'taxonomicgroup': 'Birds'},
 {'scientificname': 'Vulpes velox',
  'commonname': 'Swift Fox',
  'statelist_2005': 'Colorado,Kansas,Montana,Nebraska,New Mexico,North Dakota,Oklahoma,South Dakota,Texas,Wyoming',
  'statelist_2

15 WLCI species names appear on the Wyoming SGCN 2005 and/or 2015 list. These species names, along with additional information, are cached as a separate list from the SGCN cached document that include WLCI matched species names that appear on the lists of states other than Wyoming. 

In [16]:
# Cache the array of retrieved documents and return/display a random sample for verification
display(bis_utils.doc_cache("../cache/wy_sgcn.json", wyoming_sgcn))

{'Doc Cache File': '../cache/wy_sgcn.json',
 'Number of Documents in Cache': 25,
 'Document Number 12': {'scientificname': 'Urocitellus armatus',
  'commonname': 'Uinta Ground Squirrel',
  'statelist_2005': 'Wyoming',
  'statelist_2015': 'Wyoming',
  'matchmethod': 'Followed Accepted TSN',
  'taxonomicgroup': 'Mammals'}}

Check to see if any invalid/not accepted ITIS specie names matched with SGCN records

In [17]:
# Open up the cached updated ITIS list with invalid/not accepted species names from ITIS Exploration.ipynb
with open("../cache/invalid_itis.json", "r") as f:
    invalid_itis = json.loads(f.read())

In [18]:
# Use joblib to run multiple requests for SGCN documents in parallel via the invalid/not accepted ITIS scientific name
invalid_result = Parallel(n_jobs=8)(delayed(sgcn.search)(name) for name in [r["scientific_name"] for r in invalid_itis]) 

In [19]:
# Filter to give just cases where SGCN species names matched with invalid/not accepted ITIS names
invalid_result=[i for i in invalid_result if i['processing_metadata']['status'] == 'success']

No SGCN records macthed with the invalid/not accepted ITIS specie names

In [20]:
len(invalid_result)

0